In [ ]:
#3단계 여러페이지 크롤링하는 법 실습 + 데이터를 리스트 형태로 저장
import requests
from bs4 import BeautifulSoup
import pandas as pd

data = []
for i in range(1,5):
    response = requests.get("https://startcoding.pythonanywhere.com/basic?page{i}")#response 응답객체 데이터+명령어를 모두가지는 자료형
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    items = soup.select(".product")

    for item in items:
        category = item.select_one(".product-category").text
        name = item.select_one(".product-name").text
        link = item.select_one(".product-name > a").attrs['href']
        price = item.select_one(".product-price").text.split('원')[0].replace(',','') #strip 앞뒤 공백제거, replace는 원하는값 대체
        print(category,name,link,price)
        data.append([category,name,link,price])

In [ ]:
#데이터 프레임 만들기
df = pd.DataFrame(data, columns=['카테고리','상품명','링크','가격'])
df

In [ ]:
#엑셀 저장
df.to_excel('result.xlsx', index = False) # index = false는 첫열의 자동 번호부여를 지우는 용도임.

In [ ]:
#1단계 한개의 상품 크롤링 실습(CES 사이트)- 추가 Data 수집 확인
import requests
from bs4 import BeautifulSoup

response = requests.get("https://www.ces.tech/innovation-awards/2025/sl-22-high-resolution-spad-sensor-for-lidar/")
html = response.text
soup = BeautifulSoup(html,'html.parser')

description = soup.select_one(".rich-text").text.strip() if soup.select_one(".rich-text") else "Description not found"
prizes = soup.select_one(".mt-8.first\:mt-0.f-body-2").text.strip() if soup.select_one(".mt-8.first\:mt-0.f-body-2") else "Prizes not found"
company_des = soup.select_one(".mt-16.text-secondary.f-body-2").text.strip() if soup.select_one(".mt-16.text-secondary.f-body-2") else "Company_des not found"
booths = soup.select_one(".mt-4.first\:mt-0").text.strip() if soup.select_one(".mt-4.first\:mt-0") else "Booths not found"
booths_tag = soup.select_one(".mt-4.first\:mt-0 > a")
booths_link = booths_tag.attrs['href'] if booths_tag and 'href' in booths_tag.attrs else "Booths_link not found"
image_link = soup.select_one(".flex.justify-center.items-center.h-full.aspect-16\/9 img")

print(image_link) #description, prizes, company_des, booths, booths_link, image_link

In [ ]:
#3단계 여러페이지 크롤링하는 법 실습(CES)+이미지 저장
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import logging
import os
from openpyxl import Workbook
from openpyxl.drawing.image import Image  # 이미지 삽입을 위한 모듈
from openpyxl.utils.dataframe import dataframe_to_rows
from PIL import Image as PILImage

# 안전한 파일 이름으로 변환하는 함수
def sanitize_filename(filename):
    # Windows에서 허용되지 않는 문자를 다른 문자로 대체
    return re.sub(r'[<>:"/\\|?*]', '_', filename)

# 이미지 다운로드 및 변환 함수
def download_and_convert_image(image_url, save_path):
    try:
        response = requests.get(image_url, stream=True)
        if response.status_code == 200:
            # 원본 이미지 저장
            temp_path = "temp.webp"  # 임시로 .webp로 저장
            with open(temp_path, 'wb') as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)

            # .webp를 .jpg로 변환
            with PILImage.open(temp_path) as img:
                img.convert("RGB").save(save_path, "JPEG")  # .jpg 형식으로 저장

            # 임시 파일 삭제
            os.remove(temp_path)
            return save_path
        else:
            return None
    except Exception as e:
        logging.error(f"Error downloading or converting image {image_url}: {e}")
        return None

# Booths 번호 추출 및 포맷 함수
def extract_booths_number_and_format(booths_link):
    try:
        match = re.search(r'booth~([A-Za-z0-9%]+)', booths_link)
        if match:
            extracted_text = match.group(1)
            return extracted_text.replace('%', ' ')  # "%"를 빈칸으로 대체
        else:
            return "Booth number not found"
    except Exception as e:
        return f"Error extracting booth number: {str(e)}"

#제품 링크에서 추가 정보를 가져오는 함수
def get_additional_info(product_link):
    try:
        response = requests.get(product_link)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # 제품 설명 (HTML 구조에 맞게 CSS 선택자 수정 필요)
            description = soup.select_one(".rich-text")
            description = description.text.strip() if description else "Description not found"
            
            # 제품 사양 (HTML 구조에 맞게 CSS 선택자 수정 필요)
            prizes = soup.select_one(".mt-8.first\:mt-0.f-body-2")
            prizes = prizes.text.strip() if prizes else "Prizes not found"

            # 기업설명 (HTML 구조에 맞게 CSS 선택자 수정 필요)
            company_des = soup.select_one(".mt-16.text-secondary.f-body-2")
            company_des = company_des.text.strip() if company_des else "Company_des not found"

            # Booths (HTML 구조에 맞게 CSS 선택자 수정 필요)
            booths = soup.select_one(".mt-4.first\:mt-0")
            booths = booths.text.strip() if booths else "Booths not found"

            # Booths_link (HTML 구조에 맞게 CSS 선택자 수정 필요)
            booths_tag = soup.select_one(".mt-4.first\:mt-0 > a")
            booths_link = booths_tag.attrs['href'] if booths_tag and 'href' in booths_tag.attrs else "Booths_link not found"

            # 
            image_tag = soup.select_one(".relative.flex.flex-col.h-full.p-16.md\:pb-24.rounded-lg.bg-layer.text-primary.transition.group\/card img")
            image_url = image_tag.attrs.get('src', "Image not found") if image_tag else "Image not found"

            # 상대 경로일 경우 절대 경로로 변환
            if image_url != "Image not found" and not image_url.startswith('http'):
                image_url = f"https://www.ces.tech{image_url}"

            return description, prizes, company_des, booths, booths_link, image_url
        else:
            return "Description not found", "Prizes not found", "company_des not found", "Booths not found", "Booths_link not found", "Image not found"
    except Exception as e:
        return "Error retrieving info", str(e)
    
# 이미지 저장 경로가 없으면 디렉토리 생성하는 함수
def create_directory_for_image(image_filename):
    # 이미지 파일 경로에 해당하는 디렉토리가 없으면 생성
    directory = os.path.dirname(image_filename)
    if not os.path.exists(directory):
        os.makedirs(directory)

#모든 페이지에서 코롤링한 데이터를 저장할 리스트
all_data = []
images_dir = "images"  # 이미지 저장 폴더
os.makedirs(images_dir, exist_ok=True)

# Excel Workbook 생성
wb = Workbook()
ws = wb.active
ws.title = "CES Innovation Awards"
columns = ["Company", "Product", "Category", "Link", "Description", "Prizes", "Company Description", "Booths", "Booths Link", "Booth Number", "Image"]
ws.append(columns)

for i in range(1,24):
    try:
        response = requests.get(f"https://www.ces.tech/innovation-awards/?page={i}")#response 응답객체 데이터+명령어를 모두가지는 자료형
        html = response.text
        soup = BeautifulSoup(html,'html.parser')
        items = soup.select(".relative.flex.flex-col.h-full.p-16.md\:pb-24.rounded-lg.bg-layer.text-primary.transition.group\/card")

        for item in items:
            try:
                company = item.select_one(".outline-none.f-heading-4").text.strip()
                product = item.select_one(".mt-4.f-body-3").text.strip()
                category = item.select_one(".relative.flex.flex-wrap.gap-y-8.gap-x-4.mt-auto.pt-space-6.z-2").text.strip() #strip 앞뒤 공백제거, replace는 원하는값 대체
                link = item.select_one("a").attrs['href']

                # 제품 링크에서 추가 정보 가져오기
                full_link = f"https://www.ces.tech{link}" #상대 URL을 절대 URL로 변환

                # 제품 링크에서 추가 정보 가져오기
                description, prizes, company_des, booths, booths_link, image_url = get_additional_info(full_link)  # 추가 정보 가져오기

                # Booths 번호 추출 및 포맷
                booths_number = extract_booths_number_and_format(booths_link)

                # 이미지 URL을 안전한 파일 이름으로 처리
                if image_url != "Image not found":
                    # 이미지 파일 이름을 안전하게 처리
                    image_filename = os.path.join(images_dir, sanitize_filename(f"{company}_{product}.jpg"))
                    # 디렉토리가 없으면 생성
                    create_directory_for_image(image_filename)
                    # 이미지 다운로드 및 저장
                    image_path = download_and_convert_image(image_url, image_filename)
                else:
                    image_path = None

                # Excel에 추가할 데이터
                row = [company, product, category, full_link, description, prizes, company_des, booths, booths_link, booths_number]

                # Excel에 데이터 추가
                ws.append(row)

                # 이미지가 있으면 Excel에 삽입
                if image_path:
                    current_row = ws.max_row  # 현재 행 번호 추출
                    img = Image(image_path)
                    img.width = 100  # 이미지 크기 조정
                    img.height = 100
                    img.anchor = f"K{current_row}"  # 이미지 삽입할 셀 지정
                    ws.add_image(img)

                # 요청 간에 시간 지연 추가 (서버 과부하 방지)
                time.sleep(0.1)  # 0.1초 대기

            except Exception as e:
                logging.error(f"Error processing item on page {i}: {e}")
                
    except Exception as e:
        logging.error(f"Error processing page {i}: {e}")

# Excel 파일 저장
output_file = "CES_Innovation_Awards_with_Images.xlsx"
wb.save(output_file)
print(f"Data saved to {output_file}")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import logging
import os
from openpyxl import Workbook
from openpyxl.drawing.image import Image  # 이미지 삽입을 위한 모듈
from openpyxl.utils.dataframe import dataframe_to_rows
from PIL import Image as PILImage

# 안전한 파일 이름으로 변환하는 함수
def sanitize_filename(filename):
    # Windows에서 허용되지 않는 문자를 다른 문자로 대체
    return re.sub(r'[<>:"/\\|?*]', '_', filename)

# 이미지 다운로드 및 변환 함수
def download_and_convert_image(image_url, save_path):
    try:
        response = requests.get(image_url, stream=True)
        if response.status_code == 200:
            # 원본 이미지 저장
            temp_path = "temp.webp"  # 임시로 .webp로 저장
            with open(temp_path, 'wb') as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)

            # .webp를 .jpg로 변환
            with PILImage.open(temp_path) as img:
                img.convert("RGB").save(save_path, "JPEG")  # .jpg 형식으로 저장

            # 임시 파일 삭제
            os.remove(temp_path)
            return save_path
        else:
            return None
    except Exception as e:
        logging.error(f"Error downloading or converting image {image_url}: {e}")
        return None

# Booths 번호 추출 및 포맷 함수
def extract_booths_number_and_format(booths_link):
    try:
        match = re.search(r'booth~([A-Za-z0-9%]+)', booths_link)
        if match:
            extracted_text = match.group(1)
            return extracted_text.replace('%', ' ')  # "%"를 빈칸으로 대체
        else:
            return "Booth number not found"
    except Exception as e:
        return f"Error extracting booth number: {str(e)}"

# 제품 링크에서 추가 정보를 가져오는 함수
def get_additional_info(product_link):
    try:
        response = requests.get(product_link)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # 제품 설명 (HTML 구조에 맞게 CSS 선택자 수정 필요)
            description = soup.select_one(".rich-text")
            description = description.text.strip() if description else "Description not found"
            
            # 제품 사양 (HTML 구조에 맞게 CSS 선택자 수정 필요)
            prizes = soup.select_one(".mt-8.first\:mt-0.f-body-2")
            prizes = prizes.text.strip() if prizes else "Prizes not found"

            # 기업설명 (HTML 구조에 맞게 CSS 선택자 수정 필요)
            company_des = soup.select_one(".mt-16.text-secondary.f-body-2")
            company_des = company_des.text.strip() if company_des else "Company_des not found"

            # Booths (HTML 구조에 맞게 CSS 선택자 수정 필요)
            booths = soup.select_one(".mt-4.first\:mt-0")
            booths = booths.text.strip() if booths else "Booths not found"

            # Booths_link (HTML 구조에 맞게 CSS 선택자 수정 필요)
            booths_tag = soup.select_one(".mt-4.first\:mt-0 > a")
            booths_link = booths_tag.attrs['href'] if booths_tag and 'href' in booths_tag.attrs else "Booths_link not found"

            # 이미지 URL 추출
            image_tag = soup.select_one(".relative.flex.flex-col.h-full.p-16.md\:pb-24.rounded-lg.bg-layer.text-primary.transition.group\/card img")
            image_url = image_tag.attrs.get('src', "Image not found") if image_tag else "Image not found"

            # 상대 경로일 경우 절대 경로로 변환
            if image_url != "Image not found" and not image_url.startswith('http'):
                image_url = f"https://www.ces.tech{image_url}"

            return description, prizes, company_des, booths, booths_link, image_url
        else:
            return "Description not found", "Prizes not found", "company_des not found", "Booths not found", "Booths_link not found", "Image not found"
    except Exception as e:
        return "Error retrieving info", str(e)

# 이미지 URL 추출 함수
def get_image_url_from_item(item):
    image_tag = item.select_one(".relative.flex.flex-col.h-full.p-16.md\:pb-24.rounded-lg.bg-layer.text-primary.transition.group\/card img")
    return image_tag.attrs.get('src', "Image not found") if image_tag else "Image not found"

# 이미지 저장 경로가 없으면 디렉토리 생성하는 함수
def create_directory_for_image(image_filename):
    # 이미지 파일 경로에 해당하는 디렉토리가 없으면 생성
    directory = os.path.dirname(image_filename)
    if not os.path.exists(directory):
        os.makedirs(directory)

# 모든 페이지에서 크롤링한 데이터를 저장할 리스트
all_data = []
images_dir = "images"  # 이미지 저장 폴더
os.makedirs(images_dir, exist_ok=True)

# Excel Workbook 생성
wb = Workbook()
ws = wb.active
ws.title = "CES Innovation Awards"
columns = ["Company", "Product", "Category", "Link", "Description", "Prizes", "Company Description", "Booths", "Booths Link", "Booth Number", "Image"]
ws.append(columns)

for i in range(1, 24):
    try:
        response = requests.get(f"https://www.ces.tech/innovation-awards/?page={i}")  # response 응답객체 데이터+명령어를 모두 가짐
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        items = soup.select(".relative.flex.flex-col.h-full.p-16.md\:pb-24.rounded-lg.bg-layer.text-primary.transition.group\/card")

        for item in items:
            try:
                # 기본 정보 추출
                company = item.select_one(".outline-none.f-heading-4").text.strip()
                product = item.select_one(".mt-4.f-body-3").text.strip()
                category = item.select_one(".relative.flex.flex-wrap.gap-y-8.gap-x-4.mt-auto.pt-space-6.z-2").text.strip()
                link = item.select_one("a").attrs['href']
                full_link = f"https://www.ces.tech{link}"

                # 이미지 URL 추출
                image_url = get_image_url_from_item(item)

                # 제품 링크에서 추가 정보 가져오기
                description, prizes, company_des, booths, booths_link, image_url = get_additional_info(full_link)

                # Booths 번호 추출 및 포맷
                booths_number = extract_booths_number_and_format(booths_link)

                # 이미지 다운로드 및 저장
                image_path = None
                if image_url != "Image not found":
                    image_filename = os.path.join(images_dir, sanitize_filename(f"{company}_{product}.jpg"))
                    create_directory_for_image(image_filename)
                    image_path = download_and_convert_image(image_url, image_filename)

                # Excel에 추가할 데이터
                row = [company, product, category, full_link, description, prizes, company_des, booths, booths_link, booths_number]
                ws.append(row)

                # 이미지가 있으면 Excel에 삽입
                if image_path:
                    img = Image(image_path)
                    img.width = 100
                    img.height = 100
                    img.anchor = f"K{ws.max_row}"
                    ws.add_image(img)

                # 요청 간에 시간 지연 추가 (서버 과부하 방지)
                time.sleep(0.1)  # 0.1초 대기

            except Exception as e:
                logging.error(f"Error processing item on page {i}: {e}")

    except Exception as e:
        logging.error(f"Error processing page {i}: {e}")

# Excel 파일 저장
output_file = "CES_Innovation_Awards_with_Images.xlsx"
wb.save(output_file)
print(f"Data saved to {output_file}")


In [ ]:
import os
import requests
import time
import concurrent.futures
import pandas as pd
from bs4 import BeautifulSoup

def create_folder_if_not_exists(folder_path):
    """ 폴더가 존재하지 않으면 생성 """
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"폴더 생성됨: {folder_path}")

def create_excel_if_not_exists(file_path):
    """ 엑셀 파일이 없으면 생성 """
    if not os.path.exists(file_path):
        df = pd.DataFrame(columns=['File Name', 'URL', 'Status'])
        df.to_excel(file_path, index=False, engine='openpyxl')  # 엔진 명시
        print(f"엑셀 파일 생성됨: {file_path}")

def update_excel(file_path, file_name, url, status):
    """ 엑셀 파일에 다운로드 상태 업데이트 """
    df = pd.read_excel(file_path, engine='openpyxl')  # 엔진 명시
    new_entry = pd.DataFrame([[file_name, url, status]], columns=['File Name', 'URL', 'Status'])
    df = pd.concat([df, new_entry], ignore_index=True)
    df.to_excel(file_path, index=False, engine='openpyxl')  # 엔진 명시
    print(f"{file_name} 상태 업데이트됨: {status}")

def get_pdf_links(base_url_template, issue_range, page_range, save_folder, excel_file):
    pdf_links = []

    # 폴더 생성
    create_folder_if_not_exists(save_folder)

    # 엑셀 파일 준비
    create_excel_if_not_exists(excel_file)

    # 세션 생성
    with requests.Session() as session:
        session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Referer': 'https://www.jstage.jst.go.jp/',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
            'Connection': 'keep-alive'  # 연결 유지
        })

        # 파일 목록 가져오기
        for issue in issue_range:
            for page in page_range:
                url = base_url_template.format(issue=issue, page=page)
                print(f"접속 중: {url}")

                # 페이지 요청
                response = make_request(session, url)
                if response is None:
                    continue

                # BeautifulSoup으로 HTML 파싱
                soup = BeautifulSoup(response.text, 'html.parser')

                # 각 li 태그 내에서 PDF 링크와 파일명 추출
                for li in soup.find_all('li'):
                    # 파일명: div class="searchlist-title"
                    title_div = li.find('div', class_='searchlist-title')
                    if title_div:
                        file_name = title_div.get_text(strip=True)  # 파일명 추출

                        # 다운로드 링크: div class="lft" -> span 하위 a 태그
                        lft_div = li.find('div', class_='lft')
                        if lft_div:
                            a_tag = lft_div.find('span').find('a', href=True)
                            if a_tag and 'pdf' in a_tag['href'].lower():  # href에 pdf 포함 확인
                                pdf_link = a_tag['href']
                                full_url = pdf_link if pdf_link.startswith('http') else "https://www.jstage.jst.go.jp" + pdf_link
                                
                                # PDF 파일 다운로드 상태 기록
                                update_excel(excel_file, file_name, full_url, 'Pending')  # 대기 중 상태로 엑셀에 기록

                                pdf_links.append((full_url, file_name, save_folder))

    return pdf_links

def make_request(session, url, retries=5, delay=10):
    """ 요청을 보내고 실패시 재시도하는 함수 """
    for attempt in range(retries):
        try:
            print(f"시도 중 {attempt + 1}/{retries}: {url}")
            response = session.get(url)
            response.raise_for_status()  # 상태 코드가 200이 아니면 예외 발생
            return response
        except requests.exceptions.RequestException as e:
            print(f"요청 실패: {e}")
            if attempt < retries - 1:
                print(f"{delay}초 후 재시도...")
                time.sleep(delay)  # 재시도 전에 잠시 대기
            else:
                print("최대 재시도 횟수 초과")
    return None

def download_pdf(pdf_url, file_name, save_folder, excel_file):
    try:
        # PDF 파일명에 .pdf 확장자 추가
        file_name = file_name + ".pdf"
        save_path = os.path.join(save_folder, file_name)

        # PDF 다운로드 요청
        if os.path.exists(save_path):  # 이미 존재하면 다운로드 건너뛰기
            print(f"{file_name} 이미 존재, 건너뜁니다.")
            update_excel(excel_file, file_name, pdf_url, 'Already Downloaded')
            return

        print(f"다운로드 시작: {pdf_url}")
        response = requests.get(pdf_url, stream=True)
        if response.status_code == 200:
            with open(save_path, 'wb') as pdf_file:
                for chunk in response.iter_content(chunk_size=1024):
                    pdf_file.write(chunk)
            print(f"다운로드 완료: {save_path}")
            update_excel(excel_file, file_name, pdf_url, 'Downloaded')  # 다운로드 완료 상태로 업데이트
        else:
            print(f"다운로드 실패: {response.status_code}")
            update_excel(excel_file, file_name, pdf_url, 'Failed')  # 실패 상태로 업데이트
    except Exception as e:
        print(f"오류 발생: {e}")
        update_excel(excel_file, file_name, pdf_url, 'Failed')  # 실패 상태로 업데이트

def download_all_pdfs_in_parallel(pdf_links, excel_file):
    """ 병렬로 PDF 다운로드 """
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(download_pdf, pdf_url, file_name, save_folder, excel_file) for pdf_url, file_name, save_folder in pdf_links]
        for future in concurrent.futures.as_completed(futures):
            future.result()

# 실행 예제
base_url_template = "https://www.jstage.jst.go.jp/browse/denkiseiko/{issue}/{page}/_contents/-char/en"
issue_range = range(77, 80)  # 발행수 1~2
page_range = range(1, 5)   # 페이지 수 1~2
save_folder = "downloaded_pdfs"  # PDF를 저장할 폴더
excel_file = "download_status.xlsx"  # 엑셀 파일 경로

# PDF 링크 가져오기
pdf_links = get_pdf_links(base_url_template, issue_range, page_range, save_folder, excel_file)

# 병렬로 PDF 다운로드
download_all_pdfs_in_parallel(pdf_links, excel_file)

print("모든 PDF 다운로드 완료")


In [ ]:
import os
import requests
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from openpyxl import Workbook

# PDF 링크 및 추가 데이터 수집 함수 (page_range 제거, 페이지 자동 탐색)
def get_pdf_links(base_url_template, issue_range, excel_file, save_interval=10):
    all_data = []  # 모든 데이터를 저장할 리스트
    count = 0  # 저장 주기 카운터

    # 세션 생성
    with requests.Session() as session:
        session.headers.update({
            'User-Agent': get_random_user_agent(),
            'Referer': 'https://www.jstage.jst.go.jp/',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'
        })

        # issue별 페이지 번호 탐색
        for issue in issue_range:
            page = 1
            while True:
                url = base_url_template.format(issue=issue, page=page)
                print(f"접속 중: {url}")

                # 페이지 요청
                response = make_request(session, url)
                if response is None:
                    break  # 페이지 로드 실패 시 종료

                # BeautifulSoup으로 HTML 파싱
                soup = BeautifulSoup(response.text, 'html.parser')

                # 페이지에 내용이 없으면 종료 (더 이상 페이지가 없다는 의미)
                if not soup.find_all('li'):
                    print(f"페이지 {page}에는 더 이상 데이터가 없습니다. 탐색 종료.")
                    break

                # 각 li 태그 내에서 PDF 링크, 파일명, 저자명, 추가정보 추출
                for li in soup.find_all('li'):
                    # 파일명: div class="searchlist-title"
                    title_div = li.find('div', class_='searchlist-title')
                    if title_div:
                        file_name = title_div.get_text(strip=True)

                        # 저자명: div class="searchlist-authortags customTooltip" -> title 속성
                        author_div = li.find('div', class_='searchlist-authortags customTooltip')
                        authors = author_div['title'] if author_div and 'title' in author_div.attrs else "N/A"

                        # 추가정보: div class="searchlist-additional-info" -> <br> 기준으로 나누기
                        additional_info_div = li.find('div', class_='searchlist-additional-info')
                        additional_info_parts = []
                        if additional_info_div:
                            additional_info_parts = [part.strip() for part in additional_info_div.get_text(separator="\n").split("\n") if part.strip()]
                        
                        # 다운로드 링크: div class="lft" -> span 하위 a 태그
                        lft_div = li.find('div', class_='lft')
                        if lft_div:
                            a_tag = lft_div.find('span').find('a', href=True)
                            if a_tag and 'pdf' in a_tag['href'].lower():
                                pdf_link = a_tag['href']
                                full_url = pdf_link if pdf_link.startswith('http') else "https://www.jstage.jst.go.jp" + pdf_link

                                # 데이터 저장
                                data_entry = {
                                    'Issue': issue,
                                    'Page': page,
                                    'FileName': file_name,
                                    'Author': authors,
                                    'PDF_Link': full_url
                                }
                                # 추가정보를 각 열로 추가
                                for idx, info in enumerate(additional_info_parts):
                                    data_entry[f"Additional_Info_{idx + 1}"] = info
                                
                                all_data.append(data_entry)
                                count += 1

                                # 주기적으로 저장
                                if count % save_interval == 0:
                                    save_to_excel(all_data, excel_file)
                                    print(f"{count}개 항목 저장됨.")
                                    all_data.clear()  # 메모리 초기화

                page += 1  # 다음 페이지로 이동
                time.sleep(random.randint(2, 5))  # 랜덤 딜레이 적용

    # 남은 데이터 저장
    if all_data:
        save_to_excel(all_data, excel_file)
        print(f"마지막 데이터 {len(all_data)}개 저장 완료.")

def get_random_user_agent():
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    ]
    return random.choice(user_agents)

def make_request(session, url, retries=5, delay=10):
    """ 요청을 보내고 실패시 재시도하는 함수 """
    for attempt in range(retries):
        try:
            print(f"시도 중 {attempt + 1}/{retries}: {url}")
            response = session.get(url)
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            print(f"요청 실패: {e}")
            if attempt < retries - 1:
                sleep_time = random.randint(2, delay)
                print(f"{sleep_time}초 후 재시도...")
                time.sleep(sleep_time)
            else:
                print("최대 재시도 횟수 초과")
    return None

def save_to_excel(data, excel_file):
    """ 데이터를 엑셀 파일에 저장하는 함수 """
    df = pd.DataFrame(data)
    if not data:
        print("저장할 데이터가 없습니다.")
        return

    # 엑셀 파일이 이미 존재하면 기존 데이터와 병합
    if os.path.exists(excel_file):
        existing_df = pd.read_excel(excel_file)
        df = pd.concat([existing_df, df], ignore_index=True).drop_duplicates()

    df.to_excel(excel_file, index=False)
    print(f"데이터 저장 완료: {excel_file}")

# 실행 예제
base_url_template = "https://www.jstage.jst.go.jp/browse/denkiseiko/{issue}/{page}/_contents/-char/en"
issue_range = range(1, 39)  # 발행수 범위
excel_file = "pdf_links_with_authors3.xlsx"

get_pdf_links(base_url_template, issue_range, excel_file)
print("모든 데이터 수집 완료.")
